# Targeted Differential Expression with `shadie`
This notebook re-creates the simulations used in [manuscript]. There are only two genes on the chromosome, both of which only experience beneficial mutations. One is only expressed during the haploid life stage and the other is only expressed during the diploid life stage. 
This simulation takes advantage of advanced features of shadie, such as differential expression 

In [1]:
import shadie

## Make Custom Mutations
First, we create two mutation types with differential expression - one mutation type will only affect the haploid life stage and the other will only affect the diploid life stage. 

In [2]:
m1 = shadie.mtype(0.5, 'f', 0.1, diffexpr="haploid") #this mutation type only expressed in haploids
m2 = shadie.mtype(0.5, 'f', 0.1, diffexpr="diploid") #this mutation type only expressed in diploids

## Make Custom Genomic Elements
Here, we create two small genomic elements. One will only be expressed duriing the haploid life stage and one during the diploid life stage - this functionality is a result of the mutation types used when defining the genomic element. 

In [3]:
e1 = shadie.etype([m1], [1]) #inherits m1 __; only expressed in haploids
e2 = shadie.etype([m2], [1]) #inherits m2 ___; only expressed in diploids

## Build the Chromosome
Finally, we use our genomic elements to define genes on our chromosome. Each is 1000 base pairs long. 

In [4]:
chrom = shadie.chromosome.explicit(data = {(0, 5000000): shadie.NONCDS,
                                           (5000001, 5001000):e1,
                                           (5001001, 10000000): shadie.NONCDS,
                                           (10000001, 10001000):e2,
                                           (10001001, 15000000): shadie.NONCDS},
                                   use_synonymous_sites_in_coding=False)

In [5]:
#when we visualiize our chromosome we can see two tiny genes at 5e6 and 10e6
chrom.draw()

(<toyplot.canvas.Canvas at 0x7fb40ac11250>,
 <toyplot.mark.FillBoundaries at 0x7fb40ac5f2e0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7d6da3538f2047d286dc53bb6f1b79cc"> name: noncds
interval: (0, 5000000)
ElementType: g3
coding: False name: None
interval: (5000001, 5001000)
ElementType: g5
coding: True name: noncds
interval: (5001001, 10000000)
ElementType: g3
coding: False name: None
interval: (10000001, 10001000)
ElementType: g6
coding: True name: noncds
interval: (10001001, 15000000)
ElementType: g3
coding: False 0 2500000 5000000 7500000 10000000 12500000 15000000

## Create the Simulation 

In [7]:
with shadie.Model() as bryomono_model:
    # init the model
    bryomono_model.initialize(chromosome=chrom, 
                     sim_time = 10000, 
                     mutation_rate = 1e-6,
                     recomb_rate = 1e-07,
                     file_out = "target-bryo-mono.trees")

    bryomono_model.reproduction.bryophyte_monoicous(
        spo_pop_size=3000, 
        gam_pop_size=3000,
        gam_clone_rate=0.2,
        gam_clones_per=1,
        spo_spores_per=10,
        gam_archegonia_per=10,
        
    )
#Uncomment line below to view the script
#print(bryomono_model.script)


initialize() {

  // model type
  initializeSLiMModelType("nonWF");

  // config
  initializeRecombinationRate(1e-07, 15000000);
  initializeMutationRate(1e-06);
  initializeTreeSeq(simplificationInterval=200);

  // MutationType init
  initializeMutationType('m1', 0.5, 'f', 0.0);
    m1.convertToSubstitution = T;
  initializeMutationType('m4', 0.5, 'f', 0.1);
    m4.convertToSubstitution = T;
  initializeMutationType('m5', 0.5, 'f', 0.1);
    m5.convertToSubstitution = T;
  c(m1, m5, m4).haploidDominanceCoeff = 1.0;

  // ElementType init
  initializeGenomicElementType('g3', m1, 1);
  initializeGenomicElementType('g5', m4, 1);
  initializeGenomicElementType('g6', m5, 1);

  // Chromosome (GenomicElement init)
  initializeGenomicElement(g3, 0, 5000000);
  initializeGenomicElement(g5, 5000001, 5001000);
  initializeGenomicElement(g3, 5001001, 10000000);
  initializeGenomicElement(g6, 10000001, 10001000);
  initializeGenomicElement(g3, 10001001, 15000000);

  // constants (Population si

In [8]:
bryomono_model.run()

KeyboardInterrupt: 

## Using the `PostSim` Module

In [ ]:
import pyslim

In [ ]:
bryomono = "/"

In [ ]:
post1 = shadie.postsim.OneSim(
    bryomono,
    chromosome=chrom, 
    mut=5e-7,
    recomb=1e-7, 
    recapitate=True, 
    ancestral_Ne=3000, 
    add_neutral_mutations=True,
)

post1.tree_sequence
#post.draw_tree_sequence(max_trees=10);